In [1]:
import os
import glob
import numpy as np
import librosa
from magenta.models.nsynth import utils
from magenta.models.nsynth.wavenet import fastgen

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [2]:
model_path = '/mnt/data/Birdman/models/wavenet-ckpt/model.ckpt-200000'
sample_size = 32000
batch_size = 50

In [3]:
def rolling_window(a, window, overlap, copy = False):
    sh = (a.size - window + 1, window)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::overlap]
    if copy:
        return view.copy()
    else:
        return view
    
def batch_array(a, batch_len):
    idx = list(range(0, len(a), batch_len))
    if idx[-1] != len(a):
        idx += [len(a)]
    for start, end in zip(idx[:-1], idx[1:]):
        yield a[start:end]
        
def get_name_from_path(path):
    name = os.path.splitext(os.path.basename(path))[0]
    return name

In [5]:
recordings_paths = glob.glob('/mnt/data/Birdman/full/*.wav')
completed_names = [get_name_from_path(path) for path in glob.glob('*.npy')]
completed_names

['STHELENA-02_20140521_200000',
 'STHELENA-01_20140113_210000',
 'STHELENA-02_20140105_210100',
 'VN870508',
 'STHELENA-01_20140115_210000',
 'STHELENA-01_20140120_210514',
 'VN860037',
 'VN860045',
 'STHELENA-02_20140122_210100',
 'VN860035',
 'VN860040',
 'STHELENA-01_20140101_210000',
 'STHELENA-03_20140209_210000',
 'STHELENA-03_20140228_210000',
 'VN870488',
 'STHELENA-03_20140111_210000',
 'STHELENA-03_20140210_210000',
 'STHELENA-03_20140306_210516',
 'VN860039',
 'STHELENA-02_20140127_210100',
 'STHELENA-03_20140123_210000',
 'STHELENA-03_20140114_210000',
 'STHELENA-02_20131231_210100',
 'STHELENA-02_20140124_210100',
 'VN860042',
 'STHELENA-03_20140311_210516',
 'STHELENA-03_20140517_200000',
 'STHELENA-02_20140110_210100',
 'VN870515',
 'STHELENA-01_20140112_210000',
 'VN870507',
 'VN870486',
 'STHELENA-03_20140317_210516',
 'STHELENA-02_20140519_200000',
 'STHELENA-02_20140112_210100',
 'VN870490',
 'STHELENA-01_20140114_210000',
 'STHELENA-02_20140120_210100',
 'STHELENA-0

In [7]:
d = {}
encodings = []

for path in recordings_paths:
    name = get_name_from_path(path)
    if name in completed_names:
        print('Skipping', name)
    else:
        print('Processing', name)
        y, sr = librosa.load(path, sr=None)
        samples = rolling_window(y, sample_size, sample_size // 2)
        for sample in batch_array(samples, batch_size):
            encoding = fastgen.encode(wav_data=sample, checkpoint_path=model_path, sample_length=sample_size)
            encodings.append(encoding)
        full_enc = np.vstack(encodings)
        d[name] = full_enc
        np.save(name, full_enc)

Skipping STHELENA-02_20140110_210100
Skipping STHELENA-03_20140228_210000
Skipping STHELENA-03_20140309_210516
Skipping STHELENA-03_20140111_210000
Skipping STHELENA-03_20140209_210000
Skipping STHELENA-03_20140311_210516
Skipping STHELENA-01_20140114_210000
Skipping STHELENA-02_20140112_210100
Skipping STHELENA-02_20140521_200000
Skipping VN870486
Skipping STHELENA-01_20140113_210000
Skipping STHELENA-02_20140519_200000
Skipping STHELENA-03_20140211_210000
Skipping VN870490
Skipping STHELENA-02_20140120_210100
Skipping STHELENA-02_20140124_210100
Skipping VN860037
Skipping STHELENA-02_20140102_210100
Skipping STHELENA-02_20140127_210100
Skipping STHELENA-03_20140315_210516
Skipping STHELENA-03_20140210_210000
Skipping STHELENA-03_20140517_200000
Skipping VN860042
Skipping STHELENA-03_20140317_210516
Skipping STHELENA-01_20140120_210514
Skipping STHELENA-02_20140105_210100
Skipping STHELENA-01_20140115_210000
Skipping VN870507
Skipping VN870513
Skipping STHELENA-01_20140112_210000
Skip

OSError: Not enough free space to write 1857928704 bytes

In [ ]:
d